## in order to correctly build orbitals we must train for angular PINNS ##

TRaing below from pablo for angular

In [2]:
import torch
import torch.nn as nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import numpy as np
import matplotlib.pyplot as plt

ImportError: /home/troche2002/.local/lib/python3.10/site-packages/torch/lib/../../nvidia/cusparselt/lib/libcusparseLt.so.0: cannot read file data

In [ ]:
print(device)

In [ ]:
class PINN_R_to_R(nn.Module):
    def __init__(self, neurons_for_each_hidden_layer, activation_function):
        super().__init__()
        L = len(neurons_for_each_hidden_layer)
        layers = []

        layers.append(nn.Linear(1, neurons_for_each_hidden_layer[0])) #connect the input layer to the first hidden layer
        layers.append(activation_function) #apply the activation function to the output of the first hidden layer

        for i in range(1,L):
            layers.append(nn.Linear(neurons_for_each_hidden_layer[i-1], neurons_for_each_hidden_layer[i]))
            layers.append(activation_function)

        #connect the last hidden layer to the output layer - there is no activation function on the output
        #layer as this is the raw solution
        layers.append(nn.Linear(neurons_for_each_hidden_layer[L-1], 1))

        #nn.Sequential() constructs a neural network with all the information stored in the list layers
        self.network = nn.Sequential(*layers)

    #calculates the neural network solution
    def forward(self, x):
    # x is PINN input in [-1,1], internally you can scale if needed
      if x.dim() == 1:
        x = x.unsqueeze(-1)
      else:
        x = x.reshape(-1, 1)
      return self.network(x)


In [ ]:
l = 3
ml = 0

In [ ]:
def differentiate(outputs, inputs):
    grads = torch.autograd.grad(outputs, inputs, grad_outputs=torch.ones_like(outputs),
                                 create_graph=True, allow_unused=False)[0]
    return grads

In [ ]:
def loss_function(solution, solution_should_be):
        mse_cost_function = torch.nn.MSELoss()
        cost = mse_cost_function(solution, solution_should_be)
        return cost

associated Legendre equation

In [ ]:
def residual(y, x):
    dy_dx = differentiate(y, x)
    ddy_dxx = differentiate(dy_dx, x)

    # Ensure all tensors have shape (N, 1) for consistent broadcasting
    dy_dx = dy_dx.reshape(-1, 1)
    ddy_dxx = ddy_dxx.reshape(-1, 1)

    if x.dim() == 1:
        x = x.reshape(-1, 1)

    term1 = (1 - x**2) * ddy_dxx
    term2 = -2*x * dy_dx
    term3 = (l*(l+1) - ml*ml/(1-x**2)) * y

    return term1 + term2 + term3

def residual_loss(residual):
    residual_should_be = torch.zeros_like(residual).to(device)
    return loss_function(residual, residual_should_be)

In [ ]:
def boundary_loss(y_0, y_f):
    loss = y_0.pow(2).mean() + y_f.pow(2).mean()
    return loss

In [ ]:
def norm_loss(y,x):
    # norm = torch.trapz(y.squeeze()**2, x.squeeze())
    # return (norm - 1)**2
    return ((y.squeeze()**2).mean() - 1.2)**2

Training loop - returns trained PINN for angular

In [ ]:
def Train_1D(iterations, num_collocation_points):
    p = PINN_R_to_R([200,200,200,200], nn.Tanh())
    p = p.to(device)

    optimizer = torch.optim.Adam(p.parameters(), lr = 1e-5)

    print(f"Loss at each epoch due to")
    print(f"Epoch \t Residual \t\t Boundary \t\t Norm \t\t Total loss")

    for epoch in range(iterations):
        optimizer.zero_grad()
        if epoch % 5_000 == 0:
            x_col = 1.9999 * torch.rand(num_collocation_points, requires_grad = True, device = device) - 1.0
            x_0 = torch.tensor([-0.9999], requires_grad = True,device = device)
            x_f = torch.tensor([0.9999], requires_grad = True,device = device)

        #calculate solution
        collocation_solution = p(x_col)
        collocation_residual = residual(collocation_solution, x_col)
        y_0 = p(x_0)
        y_f = p(x_f)

        #calculate loss
        res_loss = residual_loss(collocation_residual) / num_collocation_points
        bound_loss = boundary_loss(y_0,y_f)
        norm_term = norm_loss(collocation_solution,x_col)
        loss = res_loss + bound_loss + norm_term

        #print data
        if epoch % 100 == 0:
            print(f"{epoch} \t {res_loss.item():.8f} \t\t {bound_loss.item():.8f} \t\t {norm_term.item():.8f} \t\t {loss.item():.8f}")

        if epoch % 10_000 == 0:
            for g in optimizer.param_groups:
                g['lr'] = g['lr'] * 0.9

        loss.backward(retain_graph = True)
        optimizer.step()

    return p


In [ ]:
%%time
p = Train_1D(iterations = 5_000, num_collocation_points = 1000)

## Testing the performance of PINN training ##

assesing againts theoretical solution for the theta

In [ ]:
def sol_plotter_R_to_R(x, y, title):
    plt.plot(x.cpu().detach().numpy(), y.cpu().detach().numpy())
    plt.xlabel("$x$")
    plt.ylabel("$y$")
    plt.title(title)

In [ ]:

x_test = torch.linspace(-0.999, 0.999, 500, requires_grad=True, device=device)
p_test = p(x_test)


In [ ]:
sol_plotter_R_to_R(x_test, p_test, "Solution")

In [ ]:
xactual = np.linspace(-0.999,0.999,500)
def f(x):
    return -3*x*np.sqrt(1-x**2)
yactual = f(xactual)

plt.plot(xactual,yactual)
plt.show()

In [ ]:
plt.plot(x_test.cpu().detach().numpy(), p_test.cpu().detach().numpy())
plt.plot(xactual,yactual)
plt.xlabel("$x$")
plt.ylabel("$y$")
plt.title("Solution vs Actual")

## Building orbitals ##

1. Pull the angular part solutions from the PINN
2. determine the radial solutions analyticly
3. combine the solutiosn for a completed spherical solution for the hydrogen atom
4. build the density feild for mapping to teh cartesian cordinates
5. plot the final graph for the various hydrogen orbitals



### Pull the angular part solutions from the PINN

getting the trained solutions for the theta (angular) and prepare for processing

**Angular Part (from PINN or analytical)**
$$Y_\ell^m(\theta, \phi) = N_{\ell m} \, P_\ell^m(\cos\theta) \, e^{i m \phi}$$




In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import genlaguerre, factorial
from skimage import measure

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# The analytical solution funtion for the angular
def pinn_angular(theta, p):

    # Flatten theta for input to PINN
    theta_flat = torch.tensor(theta.flatten(), dtype=torch.float32, device=device)
    x = torch.cos(theta_flat)

    with torch.no_grad():
        val = p(x).cpu().numpy()

    # Reshape back to theta's original shape
    return val.reshape(theta.shape)

### Determine the radial solutions analyticly

Using an analytic way to solve for the radial solution

**Radial Part (analytic hydrogen solution)**
$$R_{n\ell}(r) = N_{n\ell} \left( \frac{2r}{n a_0} \right)^\ell \, e^{-r/(n a_0)} \, L_{n-\ell-1}^{2\ell+1}\Big(\frac{2r}{n a_0}\Big)$$



In [ ]:
#The analytical functiomn for teh radial solutions
def R_nl(r, n, l, a0=1.0):

    rho = 2 * r / (n * a0)
    N = np.sqrt((2 / (n * a0))**3 * factorial(n - l - 1) / (2 * n * factorial(n + l)))
    L = genlaguerre(n - l - 1, 2 * l + 1)(rho)
    return N * rho**l * np.exp(-rho/2) * L

### Combine the solutiosn for a completed spherical solution for the hydrogen atom

formulate the full spherical solution

**Full 3D Wavefunction**
$$\psi_{n\ell m}(r, \theta, \phi) = R_{n\ell}(r) \, Y_\ell^m(\theta, \phi)$$



In [ ]:
 # Full probability density
# Psi2 = (Rad**2) * (Ang**2)

### Build the density feild for mapping to the cartesian cordinates

Map the full trained system onto into the cartisian cordinates

**Probability Density**
$$|\psi_{n\ell m}(r, \theta, \phi)|^2 = |R_{n\ell}(r)|^2 \, |Y_\ell^m(\theta, \phi)|^2$$

In [ ]:
def build_density(p, n, l, grid_N=60, r_max=20):

    r = np.linspace(0, r_max, grid_N)
    theta = np.linspace(0, np.pi, grid_N)
    phi = np.linspace(0, 2*np.pi, grid_N)
    R, TH, PH = np.meshgrid(r, theta, phi, indexing='ij')

    # Radial part
    Rad = R_nl(R, n, l)

    # Angular part from PINN
    Ang = pinn_angular(TH, p)
    Ang /= np.max(np.abs(Ang))  # normalize amplitude

    # Full probability density
    Psi2 = (Rad**2) * (Ang**2)

    # Convert to Cartesian coordinates
    X = R * np.sin(TH) * np.cos(PH)
    Y = R * np.sin(TH) * np.sin(PH)
    Z = R * np.cos(TH)

    return X, Y, Z, Psi2

### Plot the final graph for the various hydrogen orbitals

Building the final orbitals

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
from ipywidgets import interact, FloatSlider
import numpy as np

def plot_orbital_interactive(X, Y, Z, Psi2):

    def update(threshold=0.15):
        fig = plt.figure(figsize=(8, 8))
        ax = fig.add_subplot(111, projection='3d')

        # Mask points above threshold
        mask = Psi2 > threshold * Psi2.max()
        ax.scatter(X[mask], Y[mask], Z[mask], s=1, alpha=0.3, c='blue')

        ax.set_box_aspect([1,1,1])
        if l is not None and ml is not None:
            ax.set_title(f"Hydrogen Orbital: l={l}, m={ml}")
        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        ax.set_zlabel("Z")
        plt.show()

    # Interactive slider for threshold
    interact(update, threshold=FloatSlider(value=0.15, min=0.01, max=0.8, step=0.01, description='Threshold'))


n = l + 1
X, Y, Z, Psi2 = build_density(p, n, l)

plot_orbital_interactive(X, Y, Z, Psi2)


The trehold is a fraction of the electron density cut-off . in oother words;

- threshold = **0.1,** only points where the density is greater than **10%** of the maximum density are plotted.

- threshold = **0.5,** only points with density greater than **50%** of the maximum are plotted.